In [1]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import sqlite3
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

conn = sqlite3.connect('SF_Parking.db')
raw_folder = '/home/colin/Desktop/Parking_Project/data/raw/'
proc_folder = '/home/colin/Desktop/Parking_Project/data/processed/'

streetsweeping = gpd.read_file(raw_folder + '/street_sweeping/Street_Sweeping.shp')

In [3]:
streetsweeping[streetsweeping.cnn == '13167101']

,blockside,blocksweep,cnn,cnnrightle,corridor,district,fromhour,holidays,lf_fadd,lf_toadd,...,streetname,tohour,week1ofmon,week2ofmon,week3ofmon,week4ofmon,week5ofmon,weekday,zip_code,geometry
3512,West,1617700,13167101,L,Van Ness Ave,None,08:00,N,2501,2599,...,VAN NESS AVE,10:00,Y,N,Y,N,N,Mon,94109,LINESTRING (-122.4239821456417 37.798552453543...
3513,West,1641926,13167101,L,Van Ness Ave,None,07:00,N,2501,2599,...,VAN NESS AVE,08:00,N,Y,N,Y,N,Mon,94109,LINESTRING (-122.4239821456417 37.798552453543...
3514,West,1642258,13167101,L,Van Ness Ave,None,08:00,N,2501,2599,...,VAN NESS AVE,10:00,N,Y,N,Y,N,Mon,94109,LINESTRING (-122.4239821456417 37.798552453543...


In [6]:
streetsweeping.columns

Index(['blockside', 'blocksweep', 'cnn', 'cnnrightle', 'corridor', 'district',
       'fromhour', 'holidays', 'lf_fadd', 'lf_toadd', 'multigeom', 'nhood',
       'rt_fadd', 'rt_toadd', 'streetname', 'tohour', 'week1ofmon',
       'week2ofmon', 'week3ofmon', 'week4ofmon', 'week5ofmon', 'weekday',
       'zip_code', 'geometry'],
      dtype='object')